In [1]:
from src.preprocessor.utils.dataset_level import read_pickle, save_pickle, read_json, save_json

corpus = read_pickle('/home/thiendc/projects/legal_retrieval/data/processed/corpus_v2.pkl')
queries = read_pickle('/home/thiendc/projects/legal_retrieval/data/processed/queries_v2.pkl')
# relevant_docs = read_pickle('/home/thiendc/projects/legal_retrieval/data/processed/relevant_docs.pkl')

In [2]:
from src.preprocessor.vocab.duties_dict import DUTIES
from src.preprocessor.vocab.legal_dict import LEGAL_DICT
from src.preprocessor.vocab.special_terms import SPECIAL_TERMS

In [8]:
DUTIES =  list(DUTIES.values())
LEGAL_DICT = list(LEGAL_DICT.values())
SPECIAL_TERMS = list(SPECIAL_TERMS.values())

In [18]:
corpus1 = {i: j for i, j in list(corpus.items())[:6]}
corpus1

{'0': 'Thông_tư này hướng_dẫn tuần_tra , canh_gác bảo_vệ đê Điều trong mùa lũ đối_với các tuyến đê sông được phân_loại , phân_cấp theo quy_định tại Điều_4 của Luật Đê_Điều .',
 '1': 'Hàng năm trước mùa mưa , lũ , Ủy_ban_nhân_dân cấp xã nơi có đê phải tổ_chức lực_lượng lao_động tại địa_phương để tuần_tra , canh_gác đê và thường_trực trên các điếm_canh đê hoặc nhà dân khu_vực gần đê ( đối_với những khu_vực chưa có điếm_canh đê ) , khi có báo_động_lũ từ cấp_I trở_lên đối_với tuyến sông có đê ( sau đây gọi tắt là lực_lượng tuần_tra , canh_gác đê ) . Lực_lượng tuần_tra , canh_gác đê được tổ_chức thành các đội , do Ủy_ban_nhân_dân cấp xã ra quyết_định thành_lập ; từ 01 đến 02 kilômét đê thành_lập 01 đội ; mỗi đội có từ 12 đến 18 người , trong đó có 01 đội_trưởng và 01 hoặc 02 đội phó . Danh_sách thành_viên đội tuần_tra , canh_gác đê được niêm_yết tại điếm_canh đê thuộc địa_bàn được phân_công . Khi lũ , bão có diễn_biến phức_tạp , kéo_dài ngày , Uỷ_ban nhân_dân cấp xã có_thể quyết_định việc b

In [19]:
for i in list(corpus1.values()):
    temp = []
    for j in i.split(" "):
        if len(j.split("_")) == 3:
            if j not in DUTIES and j not in LEGAL_DICT and j not in SPECIAL_TERMS:
                if len(j.split("_")) == 3:
                    before, after = j.split("_")[0] ,"_".join(j.split("_")[1:])
                    temp.append(before)
                    temp.append(after)
                else:
                    temp.append(j)
        else:
            temp.append(j)
    temp = " ".join(temp)
    print(temp)
    print("------------------------")
    # break

Thông_tư này hướng_dẫn tuần_tra , canh_gác bảo_vệ đê Điều trong mùa lũ đối_với các tuyến đê sông được phân_loại , phân_cấp theo quy_định tại Điều_4 của Luật Đê_Điều .
------------------------
Hàng năm trước mùa mưa , lũ , Ủy_ban_nhân_dân cấp xã nơi có đê phải tổ_chức lực_lượng lao_động tại địa_phương để tuần_tra , canh_gác đê và thường_trực trên các điếm_canh đê hoặc nhà dân khu_vực gần đê ( đối_với những khu_vực chưa có điếm_canh đê ) , khi có từ cấp_I trở_lên đối_với tuyến sông có đê ( sau đây gọi tắt là lực_lượng tuần_tra , canh_gác đê ) . Lực_lượng tuần_tra , canh_gác đê được tổ_chức thành các đội , do Ủy_ban_nhân_dân cấp xã ra quyết_định thành_lập ; từ 01 đến 02 kilômét đê thành_lập 01 đội ; mỗi đội có từ 12 đến 18 người , trong đó có 01 đội_trưởng và 01 hoặc 02 đội phó . Danh_sách thành_viên đội tuần_tra , canh_gác đê được niêm_yết tại điếm_canh đê thuộc địa_bàn được phân_công . Khi lũ , bão có diễn_biến phức_tạp , kéo_dài ngày , Uỷ_ban nhân_dân cấp xã có_thể quyết_định việc bổ_s

In [11]:
save_pickle('/home/thiendc/projects/legal_retrieval/data/processed/corpus.pkl', corpus)

In [12]:
corpus_v2 = {i: j for i, j in corpus.items() if len(j.split(" ")) <= 512}

In [2]:
from src.preprocessor.preprocessor import TextPreprocessing
preprocessor =  TextPreprocessing()

In [3]:
from tqdm import tqdm
from multiprocessing import Pool

def preprocess_text_wrapper(args):
    i, j = args
    return i, preprocessor.preprocess_text(j, punctuation_remover=False, stop_word_remover=False)

# Prepare the input as a list of tuples for multiprocessing
input_data = list(queries.items())

queries_v2 = {}

# Use Pool for parallel processing and tqdm for progress bar
with Pool() as pool:
    for i, processed_text in tqdm(pool.imap(preprocess_text_wrapper, input_data), total=len(input_data), desc="Processing queries"):
        queries_v2[i] = processed_text

Processing queries: 100%|██████████| 119456/119456 [01:03<00:00, 1891.76it/s]


In [7]:
from src.preprocessor.utils.dataset_level import save_pickle
save_pickle('/home/thiendc/projects/legal_retrieval/data/processed/queries_v2.pkl', queries_v2)

In [13]:
from tqdm import tqdm
from multiprocessing import Pool

def preprocess_text_wrapper(args):
    i, j = args
    return i, preprocessor.preprocess_text(j, punctuation_remover=False, stop_word_remover=False)

# Prepare the input as a list of tuples for multiprocessing
input_data = list(corpus_v2.items())

corpus_v3 = {}

# Use Pool for parallel processing and tqdm for progress bar
with Pool() as pool:
    for i, processed_text in tqdm(pool.imap(preprocess_text_wrapper, input_data), total=len(input_data), desc="Processing queries"):
        corpus_v2[i] = processed_text

Processing queries: 100%|██████████| 234556/234556 [1:04:56<00:00, 60.19it/s] 


In [16]:
save_pickle('/home/thiendc/projects/legal_retrieval/data/processed/corpus_v2.pkl', corpus_v2)

In [24]:
jsons = read_json("/home/thiendc/projects/legal_retrieval/src/preprocessor/vocab/data/update_vocab_v2.json")
data = []
for js in jsons:
    if 'xa0' in js:
        js = js.replace("xa0", "")
        js = js.replace("xA0", "")
        data.append(js)
    else:
        data.append(js)
data = list(set(data))
save_json(data, './src/preprocessor/vocab/data/update_vocab_v2.json')